In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
vacation_data_df = pd.read_csv("weather_data/WeatherPy_vacation.csv")
vacation_data_df.head()

,City_ID,City_ID.1,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,0,0,Kodiak,US,57.79,-152.41,62.60,59,1,5.82,clear sky,0.0,0
1,1,1,Cape Town,ZA,-33.93,18.42,55.00,81,75,5.82,broken clouds,0.0,0
2,2,2,Bengkulu,ID,-3.80,102.27,75.34,86,100,4.16,overcast clouds,0.0,0
3,3,3,Punta Arenas,CL,-53.15,-70.92,33.80,86,75,9.17,broken clouds,0.0,0
4,4,4,Sao Lourenco Do Sul,BR,-31.37,-51.98,59.00,72,0,12.75,clear sky,0.0,0


In [3]:
#Create separate dataframes for each city
Valley_City_df = vacation_data_df.loc[vacation_data_df["City"] == "Valley City"]
Saint_Anthony_df = vacation_data_df.loc[vacation_data_df["City"] == "Saint Anthony"]
Fort_Atkinson_df = vacation_data_df.loc[vacation_data_df["City"] == "Fort Atkinson"]
Yankton_df = vacation_data_df.loc[vacation_data_df["City"] == "Yankton"]
Valley_City_df

,City_ID,City_ID.1,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
395,395,395,Valley City,US,46.92,-98.0,77.0,57,40,1.59,scattered clouds,0.0,0


In [4]:
#Create tuples to get lat/long pairs
Valley_City = tuple(Valley_City_df.to_numpy()[0][4:6])
Saint_Anthony = tuple(Saint_Anthony_df.to_numpy()[0][4:6])
Fort_Atkinson = tuple(Fort_Atkinson_df.to_numpy()[0][4:6])
Yankton = tuple(Yankton_df.to_numpy()[0][4:6])

In [5]:

# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

#Create the figure
fig = gmaps.figure()
Challenge_map = gmaps.directions_layer(Yankton, Fort_Atkinson, waypoints=[Saint_Anthony,Valley_City], travel_mode = "DRIVING")
fig.add_layer(Challenge_map)
fig



Figure(layout=FigureLayout(height='420px'))

In [6]:
print(Challenge_map)

Directions(data_bounds=[(42.87, -98.0), (46.92, -88.84)], end=(42.93, -88.84), layer_status='REQUEST_DENIED', start=(42.87, -97.4), stroke_color='#0088ff', waypoints=[(45.02, -93.22), (46.92, -98.0)])


In [7]:
#Add 4 cities into single DF
combined_city_df = pd.concat([Valley_City_df, Saint_Anthony_df, Fort_Atkinson_df, Yankton_df])
combined_city_df

,City_ID,City_ID.1,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
395,395,395,Valley City,US,46.92,-98.00,77.00,57,40,1.59,scattered clouds,0.0,0
69,69,69,Saint Anthony,US,45.02,-93.22,84.99,45,1,13.87,clear sky,0.0,0
287,287,287,Fort Atkinson,US,42.93,-88.84,82.99,54,51,7.00,broken clouds,0.0,0
328,328,328,Yankton,US,42.87,-97.40,78.80,78,75,11.41,light rain,0.0,0


In [8]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = combined_city_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
395,Valley City,US,77.00,scattered clouds,46.92,-98.00,
69,Saint Anthony,US,84.99,clear sky,45.02,-93.22,
287,Fort Atkinson,US,82.99,broken clouds,42.93,-88.84,
328,Yankton,US,78.80,light rain,42.87,-97.40,


In [9]:
# Set the parameters to search for a hotel in Paris.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
    "location": "48.8566, 2.3522"}
# Use base URL to search for hotels in Paris.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Make request and get the JSON data from the search.
hotels = requests.get(base_url, params=params).json()

In [10]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

In [11]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, 
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [13]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))